In [1]:
import sys
import os
import time
import json

import numpy as np
import pandas as pd

import torch as T
import torchvision


/Users/matt/.pyenv/versions/3.10.0/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw_videos = os.path.join('.', 'data', 'raw', 'videos')
raw_info = os.path.join('.', 'data', 'raw', 'info')
processed = os.path.join('.', 'data', 'processed')

transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize(224),
    torchvision.transforms.CenterCrop((224, 224)),
    torchvision.transforms.ConvertImageDtype(T.float32),
    torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]),
])

def process(n_files=None, n_frames=15, buffer=10, verbose=False):
    
    total_perf_count = time.perf_counter()
    
    if n_files is None:
        n_files = -1
    file_list = [
        os.path.splitext(filename)[0]
        for filename in os.listdir(raw_videos)[:n_files]
        if filename != '.DS_Store'
    ]
    
    segments = []
    
    for num, filename in enumerate(file_list):
        
        inner_perf_count = time.perf_counter()
        
        vid_path = os.path.join(raw_videos, filename + '.mov')
        info_path = os.path.join(raw_info, filename + '.json')
        
        with open(info_path) as f:
            json_data = json.load(f)
        
        if not any(reading['accuracy'] == 10.0 for reading in json_data['locations']):
        
            if verbose:
                if num == 0:
                    print('Finished loading file' + (len(file_list) > 1) * 's' + ':')
                print(
                    f'{time.perf_counter() - inner_perf_count:>6.3f}s ::',
                    f'{filename} -> 0 examples.',
                    f'({num+1}/{len(file_list)})'
                )
            
            continue
    
        vid, _, metadata = torchvision.io.read_video(vid_path)
        vid = T.permute(vid, (0, 3, 2, 1))
        
        vid_frame_rate = metadata['video_fps']
        vid_start_time = json_data['startTime']
        next_possible_timestamp = vid_start_time
        n_segments_from_video = 0
        
        for reading in json_data['locations']:
            if reading['timestamp'] >= next_possible_timestamp and reading['accuracy'] == 10.0:
                
                index_timestamp = round(
                    (reading['timestamp'] - vid_start_time) / 1000 * vid_frame_rate
                )
                index_end = index_timestamp + 1
                index_start = index_end - n_frames
                if index_start < 0 or index_end >= len(vid):
                    continue
                
                next_possible_timestamp = reading['timestamp'] + buffer * 1000
                n_segments_from_video += 1

                frames = vid[index_start: index_end, :, :, :]
                frames = transform(frames)
                
                segments.append((frames, T.tensor(reading['speed'], dtype=T.float)))
        
        if verbose:
            if num == 0:
                print('Finished loading file' + (len(file_list) > 1) * 's' + ':')
            print(
                f'{time.perf_counter() - inner_perf_count:>6.3f}s ::',
                f'{filename} -> {n_segments_from_video} examples.',
                f'({num+1}/{len(file_list)})'
            )
        
        if num and not num % 400:
            samples, labels = zip(*segments)
    
            samples = T.stack(samples)
            labels = T.stack(labels).reshape((-1, 1))

            T.save(
                samples,
                os.path.join(processed, 'data1', f'samples_{num}')
            )
            T.save(
                labels,
                os.path.join(processed, 'data1', f'labels_{num}')
            )
            
            segments = []
    
    if segments:
        
        samples, labels = zip(*segments)

        samples = T.stack(samples)
        labels = T.stack(labels).reshape((-1, 1))

        T.save(
            samples,
            os.path.join(processed, 'data1', f'samples_{num}_')
        )
        T.save(
            labels,
            os.path.join(processed, 'data1', f'labels_{num}_')
        )
    
    if verbose:
        elapsed = int(time.perf_counter() - total_perf_count)
        print(
            f'Done. Generated {len(segments)} examples in ' +
            f'{elapsed // 3600}:{elapsed % 3600 // 60:02}:{elapsed % 60:02}.'
        )


In [16]:
process(verbose=True)

Finished loading files:
 9.850s :: 0465ebb7-4da7a012 -> 4 examples. (1/1999)
13.598s :: 0104a3f0-9294818d -> 4 examples. (2/1999)
 0.010s :: 02f208e7-904032da -> 0 examples. (3/1999)
14.040s :: 0505c36e-579b416c -> 4 examples. (4/1999)
 0.011s :: 014070bf-4de0af14 -> 0 examples. (5/1999)
13.078s :: 03d75135-9aa0cdd3 -> 4 examples. (6/1999)
11.757s :: 041bf7e5-c5703bb5 -> 4 examples. (7/1999)
13.373s :: 02480aee-cb6bb3a5 -> 4 examples. (8/1999)
13.401s :: 02494b7b-589045ab -> 2 examples. (9/1999)
13.785s :: 024ac038-8a8b481c -> 4 examples. (10/1999)
14.580s :: 04e47e69-f7459753 -> 4 examples. (11/1999)
 0.014s :: 02514ff2-b8e3551c -> 0 examples. (12/1999)
13.098s :: 006a4209-286a5664 -> 4 examples. (13/1999)
 0.009s :: 031df6ea-15e2b64a -> 0 examples. (14/1999)
10.988s :: 01bfe142-2d54e71b -> 4 examples. (15/1999)
 0.010s :: 03d75135-83a7f6a0 -> 0 examples. (16/1999)
12.308s :: 04df85b9-abdaa88f -> 4 examples. (17/1999)
 0.011s :: 0048f391-eae6a189 -> 0 examples. (18/1999)
 0.011s :: 01

 9.594s :: 042447dc-247b04fa -> 4 examples. (152/1999)
10.894s :: 02f94134-4bd5b308 -> 4 examples. (153/1999)
12.755s :: 03d75135-70681f68 -> 4 examples. (154/1999)
 9.929s :: 015322bd-f004369e -> 4 examples. (155/1999)
 9.600s :: 03935c73-6fad10a2 -> 4 examples. (156/1999)
 9.451s :: 010b0600-47a9e3e0 -> 4 examples. (157/1999)
 9.451s :: 02ddd956-3bfcf83e -> 4 examples. (158/1999)
 9.708s :: 04dc105d-eeb828c9 -> 4 examples. (159/1999)
 7.938s :: 0554d0e1-4cfbe343 -> 3 examples. (160/1999)
 9.382s :: 017078a5-baa7a0e4 -> 1 examples. (161/1999)
 9.334s :: 04b5abe5-52cfa9ae -> 4 examples. (162/1999)
 9.171s :: 0248ab56-904b42ad -> 2 examples. (163/1999)
13.157s :: 002d290d-01969e7d -> 4 examples. (164/1999)
 9.718s :: 015b88d3-888cc0b1 -> 4 examples. (165/1999)
 9.269s :: 04ff2dcc-d4f48c8a -> 4 examples. (166/1999)
 9.333s :: 035821a3-1d1d5abb -> 4 examples. (167/1999)
 9.213s :: 044ce4d7-83a7c87c -> 4 examples. (168/1999)
 9.671s :: 000f157f-dab3a407 -> 4 examples. (169/1999)
 9.599s ::

 9.348s :: 050b5f87-684191b9 -> 4 examples. (301/1999)
 9.546s :: 030e8b7a-4dab7745 -> 4 examples. (302/1999)
 0.011s :: 03db7788-3dc1f477 -> 0 examples. (303/1999)
 0.007s :: 010799f0-4d81c585 -> 0 examples. (304/1999)
10.655s :: 0501a510-789c0ba1 -> 4 examples. (305/1999)
12.725s :: 048ae4c5-c93d4ff6 -> 4 examples. (306/1999)
 9.854s :: 0153f03b-3b26c404 -> 4 examples. (307/1999)
 9.478s :: 0070bc56-49cf077c -> 4 examples. (308/1999)
13.291s :: 001bad4e-2fa8f3b6 -> 4 examples. (309/1999)
 9.952s :: 0391b282-4d7145fe -> 4 examples. (310/1999)
 0.009s :: 01118704-2d838d7f -> 0 examples. (311/1999)
 9.537s :: 0081e3ea-2c0ec19d -> 4 examples. (312/1999)
 9.564s :: 04967fe9-4ebb0373 -> 4 examples. (313/1999)
 9.427s :: 035821a3-9f2a08bc -> 4 examples. (314/1999)
 9.586s :: 02ef72c1-e5409f23 -> 4 examples. (315/1999)
 9.224s :: 024ac038-ffa0ed50 -> 4 examples. (316/1999)
 9.742s :: 03f8899f-569acc1a -> 4 examples. (317/1999)
13.178s :: 03978fc8-6e43a30b -> 4 examples. (318/1999)
 0.010s ::

 9.435s :: 02ce41e9-147b2dd1 -> 4 examples. (450/1999)
10.968s :: 053e7ef0-17791ba1 -> 4 examples. (451/1999)
 9.375s :: 012d563d-30edbc0f -> 4 examples. (452/1999)
 0.010s :: 0199e7fc-0b6d1a35 -> 0 examples. (453/1999)
 9.310s :: 025337f4-9404c4de -> 4 examples. (454/1999)
 4.650s :: 0329b7f6-7b36589b -> 2 examples. (455/1999)
 9.169s :: 02a70c1f-6df7ce64 -> 1 examples. (456/1999)
 9.513s :: 01ebc74a-5c69af71 -> 4 examples. (457/1999)
11.441s :: 009215e2-ea063b28 -> 4 examples. (458/1999)
 0.010s :: 0407468b-d3df657c -> 0 examples. (459/1999)
 9.641s :: 052fd7cc-b5dee1de -> 4 examples. (460/1999)
 0.009s :: 00e5e793-f94de032 -> 0 examples. (461/1999)
12.071s :: 03af3912-36636dd5 -> 4 examples. (462/1999)
 0.008s :: 00313a01-97de1f42 -> 0 examples. (463/1999)
13.433s :: 00ca8821-db8033d5 -> 4 examples. (464/1999)
10.170s :: 01c5cd64-76bf9abb -> 4 examples. (465/1999)
 9.207s :: 03b8bba0-023c23ad -> 4 examples. (466/1999)
 4.759s :: 0370eddd-81051014 -> 2 examples. (467/1999)
 9.306s ::

11.772s :: 032203db-7c1e9a6f -> 4 examples. (599/1999)
 0.008s :: 04bfcebe-f0908883 -> 0 examples. (600/1999)
 0.009s :: 03e94fd1-77f26a75 -> 0 examples. (601/1999)
12.917s :: 002d290d-89d1aea8 -> 3 examples. (602/1999)
12.466s :: 04200e90-b6f3be0f -> 4 examples. (603/1999)
12.751s :: 054e6a9d-ef4a34cf -> 4 examples. (604/1999)
 9.855s :: 027a8c42-c52d5d6b -> 4 examples. (605/1999)
 0.010s :: 03793f54-c5de387b -> 0 examples. (606/1999)
 9.741s :: 016c5aac-f5e49c84 -> 4 examples. (607/1999)
 8.865s :: 033764a5-71a0152f -> 2 examples. (608/1999)
 9.412s :: 0184a4be-c7a32fbc -> 4 examples. (609/1999)
 9.204s :: 011f8481-b14725fa -> 4 examples. (610/1999)
 9.455s :: 0127cb3a-6be24a50 -> 4 examples. (611/1999)
 0.009s :: 03b0697b-79bfc82c -> 0 examples. (612/1999)
 9.339s :: 0320143b-ca3ebac1 -> 4 examples. (613/1999)
 0.009s :: 04d99a06-b17c00d8 -> 0 examples. (614/1999)
 9.479s :: 02191150-4615d36f -> 4 examples. (615/1999)
 0.010s :: 01a70d82-75ba2bd6 -> 0 examples. (616/1999)
 9.642s ::

...

 9.288s :: 010d75f8-91f41ad8 -> 1 examples. (1341/1999)
 9.250s :: 03d0bae7-be487572 -> 4 examples. (1342/1999)
 9.441s :: 01dad8ea-6732167c -> 4 examples. (1343/1999)
 9.622s :: 00d9e313-926b6698 -> 4 examples. (1344/1999)
 9.064s :: 03d8474d-948d50c6 -> 4 examples. (1345/1999)
 9.336s :: 038fbb3b-7ba9ebcc -> 4 examples. (1346/1999)
 0.009s :: 04c6b8fa-4abd5e9f -> 0 examples. (1347/1999)
 9.487s :: 04bc9f93-87d3d105 -> 4 examples. (1348/1999)
 9.222s :: 04ce1444-f2769ca7 -> 4 examples. (1349/1999)
 9.587s :: 00423717-0ef3c8dc -> 4 examples. (1350/1999)
 9.243s :: 028b5d16-45f7e11a -> 4 examples. (1351/1999)
 9.174s :: 013742f1-3a043a4e -> 4 examples. (1352/1999)
 9.556s :: 04ce1444-5230ccac -> 4 examples. (1353/1999)
10.271s :: 049885a1-cb2a82bd -> 4 examples. (1354/1999)
 9.862s :: 044798d1-ec5ec07f -> 4 examples. (1355/1999)
 9.658s :: 006c0799-964a2695 -> 4 examples. (1356/1999)
 9.712s :: 0035afff-bd191d6a -> 4 examples. (1357/1999)
 4.744s :: 023a07ea-71cd21bb -> 2 examples. (135

 9.618s :: 041bf7e5-89d32952 -> 4 examples. (1488/1999)
 9.767s :: 006c0799-10697723 -> 4 examples. (1489/1999)
 9.709s :: 0128cdde-289cd9e9 -> 4 examples. (1490/1999)
 9.526s :: 021c4a16-f4d18785 -> 4 examples. (1491/1999)
 9.347s :: 030a6a6c-fca2979d -> 4 examples. (1492/1999)
 9.805s :: 00e4cae5-c0582574 -> 4 examples. (1493/1999)
 9.767s :: 02346f91-aaa4e437 -> 4 examples. (1494/1999)
 9.355s :: 012f3428-f991fd5e -> 4 examples. (1495/1999)
 0.009s :: 01251d51-eb962582 -> 0 examples. (1496/1999)
11.072s :: 012c118a-93e8b511 -> 4 examples. (1497/1999)
 9.348s :: 0289eba3-3b2c545e -> 4 examples. (1498/1999)
 9.280s :: 02ac8427-ed7b5514 -> 2 examples. (1499/1999)
 0.010s :: 0464a98b-860229d5 -> 0 examples. (1500/1999)
11.999s :: 014e76d7-a0c222bc -> 4 examples. (1501/1999)
 4.836s :: 02346f91-27a6b98b -> 2 examples. (1502/1999)
 9.385s :: 04200e90-4a4c631e -> 4 examples. (1503/1999)
11.504s :: 0492b183-8ce155d4 -> 4 examples. (1504/1999)
11.668s :: 04c1910d-7df8b6d0 -> 4 examples. (150

 9.660s :: 00a04f65-af2ab984 -> 4 examples. (1635/1999)
 0.009s :: 03ab1996-e974d94d -> 0 examples. (1636/1999)
 9.671s :: 03e8a5e5-77d7b1d6 -> 4 examples. (1637/1999)
 9.572s :: 03026971-7ff209ef -> 4 examples. (1638/1999)
 0.010s :: 05139710-bab205bb -> 0 examples. (1639/1999)
 9.513s :: 041bf7e5-25f5dcc2 -> 4 examples. (1640/1999)
 9.728s :: 02c10fe6-dd186de7 -> 4 examples. (1641/1999)
 9.995s :: 02cc2722-4d53046c -> 4 examples. (1642/1999)
 4.972s :: 007c01ea-ad9f940b -> 2 examples. (1643/1999)
 9.450s :: 02d478d1-937f841f -> 4 examples. (1644/1999)
 0.009s :: 0332bb54-c4bee5eb -> 0 examples. (1645/1999)
 9.892s :: 029e1042-66968a89 -> 4 examples. (1646/1999)
 0.010s :: 04898289-ed683fbf -> 0 examples. (1647/1999)
 9.626s :: 03b0697b-357a37bd -> 4 examples. (1648/1999)
 9.657s :: 011f8556-ca1f4828 -> 4 examples. (1649/1999)
 9.654s :: 02c2c31c-d030d8b6 -> 2 examples. (1650/1999)
13.386s :: 044c4cac-5502747d -> 1 examples. (1651/1999)
11.381s :: 04f0228a-b833defd -> 4 examples. (165

 9.483s :: 02b30eca-c55b04c0 -> 4 examples. (1782/1999)
 9.230s :: 00067cfb-e535423e -> 4 examples. (1783/1999)
 9.624s :: 04f02ef2-e065697f -> 4 examples. (1784/1999)
 9.230s :: 0010bf16-9ee17cd9 -> 4 examples. (1785/1999)
 9.447s :: 020c83ef-ae7ac282 -> 4 examples. (1786/1999)
 4.523s :: 01f5086d-7820a48d -> 2 examples. (1787/1999)
 4.612s :: 00dc5030-237e7f71 -> 2 examples. (1788/1999)
 4.657s :: 038db692-a3659ab7 -> 2 examples. (1789/1999)
 9.155s :: 03d2cdb0-54dbfe5c -> 4 examples. (1790/1999)
 9.715s :: 020cc8f8-8b679d0b -> 4 examples. (1791/1999)
 9.856s :: 02844a1f-c957e1c3 -> 4 examples. (1792/1999)
 9.729s :: 04c20a03-45983f59 -> 4 examples. (1793/1999)
 9.438s :: 03c28a8d-9fd776c9 -> 1 examples. (1794/1999)
 0.009s :: 034fbacd-2017ee15 -> 0 examples. (1795/1999)
11.280s :: 02a39657-a59abfc7 -> 4 examples. (1796/1999)
 4.616s :: 02e06911-cb3fd71e -> 2 examples. (1797/1999)
 0.010s :: 00ccf2e8-ac055be6 -> 0 examples. (1798/1999)
 9.283s :: 047671d6-83485ddb -> 4 examples. (179

10.067s :: 03730160-cb96fffb -> 4 examples. (1930/1999)
10.700s :: 03c239e4-9e0fbb75 -> 4 examples. (1931/1999)
10.155s :: 014d895c-70b158b8 -> 4 examples. (1932/1999)
 9.066s :: 01460ec4-a1d65b66 -> 2 examples. (1933/1999)
10.044s :: 017b02cb-477b66fa -> 4 examples. (1934/1999)
10.277s :: 02006c28-dbd5e8e9 -> 4 examples. (1935/1999)
10.131s :: 00b93c6e-6298aa25 -> 4 examples. (1936/1999)
10.063s :: 02d0d9bb-3b80d771 -> 4 examples. (1937/1999)
10.081s :: 032164cf-ac289fb1 -> 4 examples. (1938/1999)
 9.721s :: 036db23c-3c184fe5 -> 4 examples. (1939/1999)
10.383s :: 02ef72c1-a37b71ce -> 4 examples. (1940/1999)
10.699s :: 04488604-03cba446 -> 4 examples. (1941/1999)
 0.009s :: 00690c26-e4bbbd72 -> 0 examples. (1942/1999)
 0.010s :: 04c40cdc-8a4350d0 -> 0 examples. (1943/1999)
 9.906s :: 02b6d2a0-54fc4ce8 -> 4 examples. (1944/1999)
10.301s :: 03565f44-411ba699 -> 4 examples. (1945/1999)
10.637s :: 02ef72c1-950d6851 -> 4 examples. (1946/1999)
10.454s :: 014c3c65-3e7539a0 -> 4 examples. (194

In [3]:
def g():
    label_list = []
    for filename in os.listdir(os.path.join(processed, 'data1')):
        if filename[:7] == 'samples':
            since = time.perf_counter()
            samples = T.load(os.path.join(processed, 'data1', filename))
            labels = T.load(os.path.join(processed, 'data1', 'labels' + filename[7:]))
            for k in range(len(labels)):
                np.save(
                    os.path.join(processed, 'data2', str(len(label_list))), 
                    samples[k].numpy()
                )
                label_list.append(labels[k])
            os.remove(os.path.join(processed, 'data1', filename))
            print(f'{time.perf_counter() - since:.3f}')
    labels_tensor = T.tensor(label_list).reshape(-1, 1)
    np.save(os.path.join(processed, 'data2_labels'), labels_tensor.numpy())
                
        

In [ ]:
g()

67.301
75.761
73.528
77.642
